In [1]:
import torch
import pandas as pd
iris_main = pd.read_csv('https://raw.githubusercontent.com/pandas-dev/pandas/master/pandas/tests/data/iris.csv')

In [33]:
iris = iris_main.copy()
iris.dtypes

SepalLength    float64
SepalWidth     float64
PetalLength    float64
PetalWidth     float64
Name            object
dtype: object

In [34]:
iris.sample(10)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
120,6.9,3.2,5.7,2.3,Iris-virginica
41,4.5,2.3,1.3,0.3,Iris-setosa
44,5.1,3.8,1.9,0.4,Iris-setosa
91,6.1,3.0,4.6,1.4,Iris-versicolor
5,5.4,3.9,1.7,0.4,Iris-setosa
56,6.3,3.3,4.7,1.6,Iris-versicolor
78,6.0,2.9,4.5,1.5,Iris-versicolor
137,6.4,3.1,5.5,1.8,Iris-virginica
49,5.0,3.3,1.4,0.2,Iris-setosa
20,5.4,3.4,1.7,0.2,Iris-setosa


In [35]:
def mean_std_table(dataframe):
    mean_std_dict = {}
    for each_column, each_dtype in zip(iris.columns, iris.dtypes):
        if each_dtype == 'float64':
            column_mean = iris[each_column].mean()
            column_std = iris[each_column].std()
            mean_std_dict[each_column] = column_mean, column_std
            dataframe[each_column] = (dataframe[each_column]- column_mean)/(column_std**2)
    return mean_std_dict

In [36]:
my_stats_dict = mean_std_table(iris)
my_stats_dict

{'SepalLength': (5.843333333333335, 0.8280661279778629),
 'SepalWidth': (3.0540000000000007, 0.4335943113621737),
 'PetalLength': (3.7586666666666693, 1.7644204199522617),
 'PetalWidth': (1.1986666666666672, 0.7631607417008414)}

In [37]:
import numpy as np
def my_hot_encoding(dataframe, feature_list_to_encode):
    encoding_dict = {}
    for each_feature in feature_list_to_encode:
        feature_dict = {}
        for index_type, each_type in enumerate(dataframe[each_feature].unique()):
            feature_dict[each_type] = index_type
        encoding_dict[each_feature] = feature_dict
    for each_feature in feature_list_to_encode:
        dataframe[each_feature] = dataframe[each_feature].map(lambda x: encoding_dict[each_feature][x])
    dataframe[each_feature].astype(np.int64)
    return encoding_dict  

In [38]:
my_discrete_dict = my_hot_encoding(iris, ['Name'])
my_discrete_dict

{'Name': {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}}

In [79]:
def even_distribution(dataframe: pd, label, my_sample_number=5, shuffle=True):
    value_count_output = dataframe[label].value_counts()
    if my_sample_number <= value_count_output.min():
        unique_output_dict = value_count_output.to_dict()
        dataframe_train_list = [dataframe[dataframe[label] == each_outcome].sample(my_sample_number) for each_outcome in unique_output_dict]
        if shuffle:
            dataframe_train_df = pd.concat(dataframe_train_list).sample(frac=1).copy()
        else:
            dataframe_train_df = pd.concat(dataframe_train_list)
        dataframe_test_df = dataframe.drop(dataframe_train_df.index).copy()
        return dataframe_train_df, dataframe_test_df
    else:
        print(value_count_output)
        print(dataframe.shape)
        print('Resorting to normal output')
#         assert my_sample_number > dataframe.shape[0]
        if shuffle:
            dataframe_train_df = dataframe.sample(my_sample_number).copy()
        else:
            dataframe_train_df = dataframe.head(my_sample_number).copy()
    return dataframe_train_df, dataframe.drop(dataframe_train_df.index).copy()

In [77]:
X_train_iris, y_test_iris = even_distribution(iris, 'Name', 60)

2    50
1    50
0    50
Name: Name, dtype: int64
(150, 5)
Resorting to normal output


In [78]:
X_train_iris.shape, y_test_iris.shape

((60, 5), (90, 5))

In [82]:
X_train_iris.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
45,-1.521574,-0.287228,-0.757639,-1.543002,0
106,-1.375736,-2.946745,0.238127,0.860785,2
100,0.665992,1.308483,0.719950,2.234377,2
73,0.374317,-1.351035,0.302370,0.002289,1
42,-2.104925,0.776579,-0.789761,-1.714701,0


In [83]:
iris_tr = torch.tensor(X_train_iris.to_numpy())

In [84]:
from torch.utils import data

# Parameters
params = {'batch_size': 7,
          'shuffle': True,
          'num_workers': 2}
training_generator = data.DataLoader(iris_tr, **params)

In [85]:
# dummy = training_generator.__iter__().next()
# dummy

tensor([[-1.3757, -2.9467,  0.2381,  0.8608,  2.0000],
        [ 1.9785, -0.2872,  0.6557,  0.6891,  2.0000],
        [ 1.2493, -2.9467,  0.6557,  1.0325,  2.0000],
        [ 0.5202, -0.8191,  0.1739,  0.1740,  1.0000],
        [ 0.2285, -0.2872,  0.3345,  1.0325,  2.0000],
        [ 0.9577, -0.2872,  0.6557,  1.7193,  2.0000],
        [-0.2090, -1.3510,  0.1096,  0.1740,  1.0000]], dtype=torch.float64)

In [299]:
# dummy[:,-1].type(torch.IntTensor)

tensor([0, 2, 1, 0, 0, 2, 0], dtype=torch.int32)

In [276]:
# w = torch.rand(dim, requires_grad=True)
w = torch.rand(4,3, requires_grad=True)
b = torch.rand(1, requires_grad=True)
w, b

(tensor([[0.3894, 0.1339, 0.0220],
         [0.9231, 0.3532, 0.4501],
         [0.5169, 0.4078, 0.2602],
         [0.6727, 0.2713, 0.5925]], requires_grad=True),
 tensor([0.0497], requires_grad=True))

In [277]:
sample_batch = dummy[:,0:-1]
sample_batch, w

(tensor([[-1.9591, -4.0106, -0.7898, -1.5430],
         [ 0.3743, -2.4148,  0.5915,  0.3457],
         [-0.2090, -2.4148, -0.0831, -0.3411],
         [-2.2508, -0.2872, -0.8540, -1.8864],
         [-1.3757,  0.2447, -0.7255, -1.8864],
         [ 2.5619, -0.2872,  0.9127,  1.5476],
         [-1.0841,  1.8404, -0.7255, -1.7147]], dtype=torch.float64),
 tensor([[0.3894, 0.1339, 0.0220],
         [0.9231, 0.3532, 0.4501],
         [0.5169, 0.4078, 0.2602],
         [0.6727, 0.2713, 0.5925]], requires_grad=True))

In [279]:
y_hat = torch.matmul(sample_batch, w.type(torch.float64))
y_hat, y_hat.shape

(tensor([[-5.9113, -2.4194, -2.9679],
         [-1.5452, -0.4678, -0.7200],
         [-2.5830, -1.0073, -1.3153],
         [-2.8519, -1.2628, -1.5186],
         [-1.9537, -0.9054, -1.2265],
         [ 2.2451,  1.0336,  1.0814],
         [-0.2516, -0.2562, -0.4002]], dtype=torch.float64,
        grad_fn=<MmBackward>),
 torch.Size([7, 3]))

In [280]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [281]:
# https://stackoverflow.com/questions/49390842/cross-entropy-in-pytorch
loss = criterion(y_hat, y_actual.type(torch.LongTensor))
loss

tensor(1.7676, dtype=torch.float64, grad_fn=<NllLossBackward>)

In [282]:
y_actual

tensor([0, 2, 1, 0, 0, 2, 0])

In [290]:
https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss
my_sum = 0
for index, value in enumerate(y_actual.type(torch.IntTensor)):
    my_sum += y_hat[index, value]*-1 + torch.log(torch.exp(y_hat[index]).sum())
print(my_sum/7)

tensor(1.7676, dtype=torch.float64, grad_fn=<DivBackward0>)


In [88]:
def my_loss(y_hat, y_actual):
    my_sum = 0
    for index, value in enumerate(y_actual.type(torch.IntTensor)):
        my_sum += y_hat[index, value]*-1 + torch.log(torch.exp(y_hat[index]).sum())
    return my_sum/y_hat.shape[0]

In [379]:
step_size = torch.tensor(.0001)
num_epochs = 5000
minibatch_size = 20
w = torch.rand(4,3, requires_grad=True, dtype=torch.float64)
b = torch.rand(1, requires_grad=True, dtype=torch.float64)
w, b
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

(tensor([[0.6673, 0.3759, 0.0758],
         [0.5282, 0.0964, 0.6912],
         [0.2048, 0.8031, 0.7101],
         [0.7284, 0.2818, 0.9685]], dtype=torch.float64, requires_grad=True),
 tensor([0.3194], dtype=torch.float64, requires_grad=True))

In [86]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [91]:
# torchvariable.to(device)
# model.to(device) Class instances can go into CUDA too
X_train_tensor = X_train_tensor.to(device)
x_test_tensor = x_test_tensor.to(device)

Y_train_tensor = Y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

NameError: name 'criterion' is not defined

In [380]:
epochs = 5000
loss_arr = []
for i in range(epochs):
    for batch in training_generator:
        y_hat = torch.matmul(batch[:,:-1], w) + b
#         loss = criterion(y_hat, batch[:,-1].type(torch.LongTensor)) #average loss per batch
        loss = my_loss(y_hat, batch[:,-1].type(torch.LongTensor))
        loss_arr.append(loss)
        loss.backward()
        w.data -= step_size * w.grad.data
        b.data -= step_size * b.grad.data
        w.grad.data.zero_()
        b.grad.data.zero_()
    if i % 25 == 0:
        print(loss)

tensor(0.8294, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.9489, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(1.0555, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(1.1955, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7960, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.5790, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.9364, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.6926, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.3345, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.5476, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.5908, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.3524, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7636, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7115, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.2724, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.7149, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.3455, dtype=torch.float64, grad

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
with torch.no_grad():

  correct = 0
  total = 0

  outputs = model(x_test_tensor)
  _, predicted = torch.max(outputs.data, 1)

  y_test = y_test_tensor.cpu().numpy()
  predicted = predicted.cpu()

  print("Accuracy: ", accuracy_score(predicted, y_test))
  print("Precision: ", precision_score(predicted, y_test, average='weighted'))
  print("Recall: ", recall_score(predicted, y_test, average='weighted'))

In [ ]:
import pickle
with open('model.pkl', 'wb') as file:
  pickle.dump(model, file)
files.download('model.pkl')